In [12]:
import pandas as pd
import os

# Specify the data folder
data_folder = "./../original_data"

# Read each CSV file into a separate DataFrame
attendance_df = pd.read_csv(os.path.join(data_folder, 'attendance.csv'))
games_df = pd.read_csv(os.path.join(data_folder, 'games.csv'))
standings_df = pd.read_csv(os.path.join(data_folder, 'standings.csv'))

# Year filtering

In [13]:
attendance_df = attendance_df[(attendance_df['year'] >= 2009) & (attendance_df['year'] <= 2019)]
games_df = games_df[(games_df['year'] >= 2009) & (games_df['year'] <= 2019)]
standings_df = standings_df[(standings_df['year'] >= 2007) & (standings_df['year'] <= 2019)] # Here we allow two years before

# Splitt regular season and other 

In [14]:
# We create a new df with this values called spetial games
special_games_df = games_df[games_df['week'].isin(['WildCard', 'Division', 'ConfChamp', 'SuperBowl'])].copy()

# We drop the special games from the original df
games_df = games_df[~games_df['week'].isin(['WildCard', 'Division', 'ConfChamp', 'SuperBowl'])].copy()

# Column types


In [15]:
# Specify column types for attendance_df
attendance_df[["year", "week", "weekly_attendance"]] = attendance_df[["year", "week", "weekly_attendance"]].astype("Int16", errors="ignore")
attendance_df["team_name"] = attendance_df["team_name"].astype("string")

# Specify column types for games_df
games_df[["year", "week", "pts_win", "pts_loss", "yds_win", "yds_loss", "turnovers_win", "turnovers_loss"]] = games_df[["year", "week", "pts_win", "pts_loss", "yds_win", "yds_loss", "turnovers_win", "turnovers_loss"]].astype("Int16", errors="ignore")
games_df[["home_team_name", "away_team_name", "home_team_city", "away_team_city"]] = games_df[["home_team_name", "away_team_name", "home_team_city", "away_team_city"]].astype("string")

# Specify column types for standings_df
standings_df[["year", "wins", "loss", "points_for", "points_against", "points_differential"]] = standings_df[["year", "wins", "loss", "points_for", "points_against", "points_differential"]].astype("Int16", errors="ignore")
standings_df[["margin_of_victory", "strength_of_schedule", "simple_rating", "offensive_ranking", "defensive_ranking"]] = standings_df[["margin_of_victory", "strength_of_schedule", "simple_rating", "offensive_ranking", "defensive_ranking"]].astype("float")
standings_df[["playoffs", "team_name", "sb_winner"]] = standings_df[["playoffs", "team_name", "sb_winner"]].astype("string")

# Specify column types for special_games_df
special_games_df[["year", "pts_win", "pts_loss", "yds_win", "yds_loss", "turnovers_win", "turnovers_loss"]] = special_games_df[["year", "pts_win", "pts_loss", "yds_win", "yds_loss", "turnovers_win", "turnovers_loss"]].astype("Int16", errors="ignore")
special_games_df[["week", "home_team_name", "away_team_name", "home_team_city"]] = special_games_df[["week", "home_team_name", "away_team_name", "home_team_city"]].astype("string")

# Create attendance as percentage

In [16]:
# Maximum of all seasons
attendance_df["weekly_attendance_ratio"] = attendance_df.groupby("team_name")["weekly_attendance"].transform(lambda x: x / x.max()) 

# Maximum for each season since they moved  the stadium
selected_teams_names = ["Chargers", "Rams"]
attendance_df_subset = attendance_df[attendance_df["team_name"].isin(selected_teams_names)].copy()
attendance_df_subset["weekly_attendance_ratio"] = attendance_df_subset.groupby(["team_name", "year"])["weekly_attendance"].transform(lambda x: x / x.max())

# Merging 
attendance_df.loc[attendance_df_subset.index, "weekly_attendance_ratio"] = attendance_df_subset["weekly_attendance_ratio"]

# Select relevant columns for attendance_df
attendance_df = attendance_df[["year", "team_name", "week", "weekly_attendance", "weekly_attendance_ratio"]]# Resort columns

In [17]:
# import matplotlib.pyplot as plt

# teams_to_plot = attendance_df["team_full_name"].unique()

# for team in teams_to_plot:
#     team_data = attendance_df[attendance_df["team_full_name"] == team]
#     plt.figure(figsize=(10, 4))
#     plt.plot(
#         pd.to_datetime(team_data["year"].astype(str) + "-W" + team_data["week"].astype(str) + "-1", format="%Y-W%W-%w"),
#         team_data["weekly_attendance"],
#         label=team
#     )
#     plt.xlabel("Date")
#     plt.ylabel("Weekly Attendance")
#     plt.title(f"Weekly Attendance Over Time: {team}")
#     plt.legend()
#     plt.tight_layout()
#     plt.show()


# Merge attendance to games 

In [18]:
games_df = pd.merge(
    games_df,
    attendance_df,
    left_on=['home_team_name', 'year', 'week'],
    right_on=['team_name', 'year', 'week'],
    how='inner'
).drop(columns=['team_name']).sort_values(by=['year', 'week', 'time'])

In [19]:
# Create previous game attendance by grouping by home_team_name and lagging
games_df = games_df.sort_values(by=["home_team_name", "year", "week"])
games_df["prev_game_attendance"] = games_df.groupby("home_team_name")["weekly_attendance"].shift(1)

In [20]:
# Calculate average attendance for each team and season
average_attendance_per_season = attendance_df.groupby(['year', 'team_name'])['weekly_attendance'].mean().reset_index()

# Rename and show average attendance per season
average_attendance_per_season.rename(columns={'weekly_attendance': 'avg_season_attendance'}, inplace=True)

# Lag avg_season_attendance by one year for each team
average_attendance_per_season['avg_season_attendance_prev'] = (
    average_attendance_per_season.groupby('team_name')['avg_season_attendance'].shift(1)
)

# Merge lagged average attendance into games_df using home_team and year
games_df = games_df.merge(
    average_attendance_per_season[['year', 'team_name', 'avg_season_attendance_prev']],
    left_on=['year', 'home_team_name'],
    right_on=['year', 'team_name'],
    how='left'
).drop(columns=['team_name'])

# Last year performance

In [21]:
prev_year_winners = standings_df[standings_df['sb_winner'] == 'Won Superbowl'][["year", "team_name"]].copy()
prev_year_winners["year"] += 1  # Shift the year forward so we can match it to the next season

# Merge it with games_df on year and home_team
games_df = games_df.merge(
    prev_year_winners,
    how="left",
    left_on=["year", "home_team_name"],
    right_on=["year", "team_name"]
)

# If the merge was successful, it means the home team was the previous year's Super Bowl winner
games_df["home_team_superbowl_winner_last_season"] = games_df["team_name"].notna()

# Drop the extra columns from the merge
games_df = games_df.drop(columns="team_name")

# Check if the away team is the previous year's Super Bowl winner
games_df = games_df.merge(
    prev_year_winners,
    how="left",
    left_on=["year", "away_team_name"],
    right_on=["year", "team_name"]
)

# If the merge was successful, it means the away team was the previous year's Super Bowl winner
games_df["away_team_superbowl_winner_last_season"] = games_df["team_name"].notna()

games_df = games_df.drop(columns="team_name")

In [22]:
# Filter teams that made the playoffs (but not necessarily won the Super Bowl)
playoffs_finishers = standings_df[standings_df['playoffs'] == 'Playoffs'][["year", "team_name"]].copy()
playoffs_finishers["year"] += 1  # Shift to next season for matching

# Super Bowl winner
prev_year_winners = standings_df[standings_df['sb_winner'] == 'Won Superbowl'][["year", "team_name"]].copy()
prev_year_winners["year"] += 1

# Merge for home team: Super Bowl winner
games_df = games_df.merge(
    prev_year_winners,
    how="left",
    left_on=["year", "home_team"],
    right_on=["year", "team_name"]
)
games_df["home_team_superbowl_winner_last_season"] = games_df["team_name"].notna()
games_df = games_df.drop(columns="team_name")

# Merge for away team: Super Bowl winner
games_df = games_df.merge(
    prev_year_winners,
    how="left",
    left_on=["year", "away_team_name"],
    right_on=["year", "team_name"]
)
games_df["away_team_superbowl_winner_last_season"] = games_df["team_name"].notna()
games_df = games_df.drop(columns="team_name")

# Merge for home team: Playoff participant
games_df = games_df.merge(
    playoffs_finishers,
    how="left",
    left_on=["year", "home_team"],
    right_on=["year", "team_name"]
)
games_df["home_team_playoffs_last_season"] = games_df["team_name"].notna()
games_df = games_df.drop(columns="team_name")

# Merge for away team: Playoff participant
games_df = games_df.merge(
    playoffs_finishers,
    how="left",
    left_on=["year", "away_team_name"],
    right_on=["year", "team_name"]
)
games_df["away_team_playoffs_last_season"] = games_df["team_name"].notna()
games_df = games_df.drop(columns="team_name")

In [23]:
# Ensure the data is sorted chronologically for correct rolling computation
games_df = games_df.sort_values(by=["year", "week"]).reset_index(drop=True)

# Create helper columns for win flags
games_df["home_win"] = (games_df["winner"] == games_df["home_team"]).astype(int)
games_df["away_win"] = (games_df["winner"] == games_df["away_team"]).astype(int)

# Build a long format DataFrame with one row per team per game
home_games = games_df[["year", "week", "home_team", "home_win"]].copy()
home_games.columns = ["year", "week", "team", "win"]

away_games = games_df[["year", "week", "away_team", "away_win"]].copy()
away_games.columns = ["year", "week", "team", "win"]

team_games = pd.concat([home_games, away_games]).sort_values(by=["team", "year", "week"]).reset_index(drop=True)

# Compute rolling win count over the last 3 games for each team
team_games["rolling_win_count_3wk"] = (
    team_games.groupby("team")["win"]
    .transform(lambda x: x.rolling(window=3, min_periods=3).sum().shift(1))  # shift(1) to exclude current week
)

# Fill NaNs with the last known value from previous year for the same team
team_games["rolling_win_count_3wk"] = (
    team_games.groupby("team")["rolling_win_count_3wk"]
    .transform(lambda x: x.ffill())
)

# Merge back into main DataFrame for home and away teams
# Home
home_rolling = team_games[["year", "week", "team", "rolling_win_count_3wk"]].copy()
home_rolling.columns = ["year", "week", "home_team", "home_team_wins_last_3"]

games_df = games_df.merge(home_rolling, on=["year", "week", "home_team"], how="left")

# Away
away_rolling = team_games[["year", "week", "team", "rolling_win_count_3wk"]].copy()
away_rolling.columns = ["year", "week", "away_team", "away_team_wins_last_3"]

games_df = games_df.merge(away_rolling, on=["year", "week", "away_team"], how="left")

games_df

,year,week,home_team,away_team,winner,tie,day,date,time,pts_win,...,prev_game_attendance,avg_season_attendance_prev,home_team_superbowl_winner_last_season,away_team_superbowl_winner_last_season,home_team_playoffs_last_season,away_team_playoffs_last_season,home_win,away_win,home_team_wins_last_3,away_team_wins_last_3
0,2009,1,Cincinnati Bengals,Denver Broncos,Denver Broncos,NaN,Sun,September 13,1:02PM,12,...,NaN,NaN,False,False,False,False,0,1,NaN,NaN
1,2009,1,Cleveland Browns,Minnesota Vikings,Minnesota Vikings,NaN,Sun,September 13,1:03PM,34,...,NaN,NaN,False,False,False,True,0,1,NaN,NaN
2,2009,1,Tampa Bay Buccaneers,Dallas Cowboys,Dallas Cowboys,NaN,Sun,September 13,1:03PM,34,...,NaN,NaN,False,False,False,False,0,1,NaN,NaN
3,2009,1,Arizona Cardinals,San Francisco 49ers,San Francisco 49ers,NaN,Sun,September 13,1:15PM,20,...,NaN,NaN,False,False,False,False,0,1,NaN,NaN
4,2009,1,Indianapolis Colts,Jacksonville Jaguars,Indianapolis Colts,NaN,Sun,September 13,1:02PM,14,...,NaN,NaN,False,False,False,False,1,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2019,17,Los Angeles Rams,Arizona Cardinals,Los Angeles Rams,NaN,Sun,December 29,4:25PM,31,...,71501.0,68900.5625,False,False,False,False,1,0,1.0,2.0
2812,2019,17,Baltimore Ravens,Pittsburgh Steelers,Baltimore Ravens,NaN,Sun,December 29,4:25PM,28,...,70545.0,65836.4375,False,False,False,False,1,0,3.0,1.0
2813,2019,17,Seattle Seahawks,San Francisco 49ers,San Francisco 49ers,NaN,Sun,December 29,8:20PM,26,...,69022.0,69645.4375,False,False,False,False,0,1,1.0,2.0
2814,2019,17,Houston Texans,Tennessee Titans,Tennessee Titans,NaN,Sun,December 29,4:25PM,35,...,71769.0,69530.0000,False,False,False,False,0,1,2.0,1.0


# Redifining pts in terms of home and away 

In [24]:
# Safer and clearer naming
games_df["pts_home"] = games_df.apply(lambda row: row["pts_win"] if row["winner"] == row["home_team"] else row["pts_loss"], axis=1)
games_df["pts_away"] = games_df.apply(lambda row: row["pts_win"] if row["winner"] == row["away_team"] else row["pts_loss"], axis=1)

games_df["yds_home"] = games_df.apply(lambda row: row["yds_win"] if row["winner"] == row["home_team"] else row["yds_loss"], axis=1)
games_df["yds_away"] = games_df.apply(lambda row: row["yds_win"] if row["winner"] == row["away_team"] else row["yds_loss"], axis=1)

games_df["turnovers_home"] = games_df.apply(lambda row: row["turnovers_win"] if row["winner"] == row["home_team"] else row["turnovers_loss"], axis=1)
games_df["turnovers_away"] = games_df.apply(lambda row: row["turnovers_win"] if row["winner"] == row["away_team"] else row["turnovers_loss"], axis=1)

# Drop the original columns that are no longer needed
games_df = games_df.drop(columns=["pts_win", "pts_loss", "yds_win", "yds_loss", "turnovers_win", "turnovers_loss"])

# Standarize with previous year values

In [25]:
# # Combine all stats: Points, Yards, Turnovers
# # Home stats
# home_stats = games_df[["year", "home_team", "pts_win", "yds_win", "turnovers_win"]].rename(
#     columns={
#         "home_team": "team",
#         "pts_win": "points",
#         "yds_win": "yds",
#         "turnovers_win": "turnovers"
#     }
# )

# # Away stats
# away_stats = games_df[["year", "away_team", "pts_loss", "yds_loss", "turnovers_loss"]].rename(
#     columns={
#         "away_team": "team",
#         "pts_loss": "points",
#         "yds_loss": "yds",
#         "turnovers_loss": "turnovers"
#     }
# )

# # Combine for per-team/year averages
# all_stats = pd.concat([home_stats, away_stats])
# avg_stats = all_stats.groupby(["team", "year"])[["points", "yds", "turnovers"]].mean().reset_index()

# # Shift year so previous season stats apply to current games
# avg_stats["year"] += 1

# # Merge previous season averages into games_df for both home and away teams
# for prefix, team_col in [("home", "home_team"), ("away", "away_team")]:
#     games_df = games_df.merge(
#         avg_stats.rename(columns={
#             "team": team_col,
#             "points": f"{prefix}_avg_prev_pts",
#             "yds": f"{prefix}_avg_prev_yds",
#             "turnovers": f"{prefix}_avg_prev_turnovers"
#         }),
#         on=[team_col, "year"],
#         how="left"
#     )

# # ----- Standardize all stats -----
# games_df["standardized_pts_win"] = games_df["pts_win"] / games_df["home_avg_prev_pts"]
# games_df["standardized_pts_loss"] = games_df["pts_loss"] / games_df["away_avg_prev_pts"]

# games_df["standardized_yds_win"] = games_df["yds_win"] / games_df["home_avg_prev_yds"]
# games_df["standardized_yds_loss"] = games_df["yds_loss"] / games_df["away_avg_prev_yds"]

# games_df["standardized_turnovers_win"] = games_df["turnovers_win"] / games_df["home_avg_prev_turnovers"]
# games_df["standardized_turnovers_loss"] = games_df["turnovers_loss"] / games_df["away_avg_prev_turnovers"]

# Z-score

In [26]:
# Prepare home team stats
home_stats = games_df[["year", "home_team_name", "pts_home", "yds_home", "turnovers_home"]].rename(
    columns={
        "home_team_name": "team",
        "pts_home": "points",
        "yds_home": "yds",
        "turnovers_home": "turnovers"
    }
)

# Prepare away team stats
away_stats = games_df[["year", "away_team_name", "pts_away", "yds_away", "turnovers_away"]].rename(
    columns={
        "away_team_name": "team",
        "pts_away": "points",
        "yds_away": "yds",
        "turnovers_away": "turnovers"
    }
)

# Combine into long format: one row per team-game
all_stats = pd.concat([home_stats, away_stats], ignore_index=True)

# Compute per-team/year means and stds
team_year_stats = (
    all_stats.groupby(["team", "year"])[["points", "yds", "turnovers"]]
    .agg(["mean", "std"])
    .reset_index()
)

# Flatten MultiIndex columns
team_year_stats.columns = [
    "team", "year",
    "points_mean", "points_std",
    "yds_mean", "yds_std",
    "turnovers_mean", "turnovers_std"
]

# Shift to apply previous year's stats to current year's games
team_year_stats["year"] += 1

# Merge into games_df for both home and away teams
for prefix, team_col in [("home", "home_team_name"), ("away", "away_team_name")]:
    stats = team_year_stats.rename(columns={
        "team": team_col,
        "points_mean": f"{prefix}_prev_pts_mean",
        "points_std": f"{prefix}_prev_pts_std",
        "yds_mean": f"{prefix}_prev_yds_mean",
        "yds_std": f"{prefix}_prev_yds_std",
        "turnovers_mean": f"{prefix}_prev_turnovers_mean",
        "turnovers_std": f"{prefix}_prev_turnovers_std"
    })
    games_df = games_df.merge(stats, on=[team_col, "year"], how="left")


# Home Z-scores
games_df["zscore_pts_home"] = (games_df["pts_home"] - games_df["home_prev_pts_mean"]) / games_df["home_prev_pts_std"]
games_df["zscore_yds_home"] = (games_df["yds_home"] - games_df["home_prev_yds_mean"]) / games_df["home_prev_yds_std"]
games_df["zscore_turnovers_home"] = (games_df["turnovers_home"] - games_df["home_prev_turnovers_mean"]) / games_df["home_prev_turnovers_std"]

# Away Z-scores
games_df["zscore_pts_away"] = (games_df["pts_away"] - games_df["away_prev_pts_mean"]) / games_df["away_prev_pts_std"]
games_df["zscore_yds_away"] = (games_df["yds_away"] - games_df["away_prev_yds_mean"]) / games_df["away_prev_yds_std"]
games_df["zscore_turnovers_away"] = (games_df["turnovers_away"] - games_df["away_prev_turnovers_mean"]) / games_df["away_prev_turnovers_std"]

In [27]:
games_df.isna().sum()

year                                         0
week                                         0
home_team                                    0
away_team                                    0
winner                                       0
tie                                       2808
day                                          0
date                                         0
time                                         0
home_team_name                               0
home_team_city                               0
away_team_name                               0
away_team_city                               0
weekly_attendance                            0
weekly_attendance_ratio                      0
prev_game_attendance                        32
avg_season_attendance_prev                 256
home_team_superbowl_winner_last_season       0
away_team_superbowl_winner_last_season       0
home_team_playoffs_last_season               0
away_team_playoffs_last_season               0
home_win     

# Drop 2009 year which was used just for having the first year as no missings, and redundant cols

In [28]:
games_df.columns

Index(['year', 'week', 'home_team', 'away_team', 'winner', 'tie', 'day',
       'date', 'time', 'home_team_name', 'home_team_city', 'away_team_name',
       'away_team_city', 'weekly_attendance', 'weekly_attendance_ratio',
       'prev_game_attendance', 'avg_season_attendance_prev',
       'home_team_superbowl_winner_last_season',
       'away_team_superbowl_winner_last_season',
       'home_team_playoffs_last_season', 'away_team_playoffs_last_season',
       'home_win', 'away_win', 'home_team_wins_last_3',
       'away_team_wins_last_3', 'pts_home', 'pts_away', 'yds_home', 'yds_away',
       'turnovers_home', 'turnovers_away', 'home_prev_pts_mean',
       'home_prev_pts_std', 'home_prev_yds_mean', 'home_prev_yds_std',
       'home_prev_turnovers_mean', 'home_prev_turnovers_std',
       'away_prev_pts_mean', 'away_prev_pts_std', 'away_prev_yds_mean',
       'away_prev_yds_std', 'away_prev_turnovers_mean',
       'away_prev_turnovers_std', 'zscore_pts_home', 'zscore_yds_home',
       'z

In [29]:
columns_to_select = ['year', 'week', 'home_team_name', 'away_team_name','day','date', 'time', 
    'weekly_attendance', 'weekly_attendance_ratio', 'prev_game_attendance', 'avg_season_attendance_prev', 'home_team_superbowl_winner_last_season',
    'away_team_superbowl_winner_last_season', 'home_team_playoffs_last_season', 
    'away_team_playoffs_last_season','home_team_wins_last_3', 'away_team_wins_last_3',
    'pts_home', 'pts_away', 'yds_home', 'yds_away', 'turnovers_home', 'turnovers_away', # I keep it for cheching this is not modified
    'zscore_pts_home', 'zscore_yds_home','zscore_turnovers_home', 'zscore_pts_away', 'zscore_yds_away','zscore_turnovers_away'
    ]

games_df = games_df[games_df["year"]>2009][columns_to_select].copy().sort_values(by=['year', 'week', 'date', 'time'])

In [30]:
# Save games_df to the process folder as a CSV file
process_folder = "./../processed_data"
os.makedirs(process_folder, exist_ok=True)
games_df.to_csv(os.path.join(process_folder, "games.csv"), index=False)

# Merging weather data (New York Giants)

In [31]:
team_name = 'Giants'
nickname = team_name.split(' ')[-1].lower()

weather_data_path = './../weather_data'
weather_df = pd.read_csv(weather_data_path+'/export_'+nickname+'.csv')

games_df['full_date'] = pd.to_datetime(games_df['date'] + ' ' + games_df['year'].astype(str), format='%B %d %Y')

df_pats = games_df.loc[games_df['home_team_name'] == team_name]

weather_df.rename(columns={'date': 'full_date'}, inplace=True)
weather_df['full_date'] = pd.to_datetime(weather_df['full_date'])

df_pats = df_pats.merge(weather_df, on=['full_date'], how='left')

process_folder = "./../datasets_w_weather_data"
os.makedirs(process_folder, exist_ok=True)
df_pats.to_csv(os.path.join(process_folder, f"{team_name}.csv"), index=False)